In [0]:
!pip -q install colorlog
!pip -q install keras_radam
!pip -q install keras_tqdm
#!pip install tensorflow==2.1.0-rc1
#!pip install tensorflow==2.2
#!pip install --upgrade tensorflow
#!pip install tensorflow-probability==0.9.0
!pip install tensorflow==2.1.0 
!pip install tensorflow_probability==0.8.0

     |████████████████████████████████| 421.8MB 31kB/s 
     |████████████████████████████████| 450kB 52.3MB/s 
     |████████████████████████████████| 3.9MB 46.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=aa0aeefbda35eef3f78656ac9deb1f1bbb28135a3261f5a827c6672375796c4d
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installatio

# Settings

In [0]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
/


In [0]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [0]:
os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..')
print(os.getcwd())

/


In [0]:
%cd /content/
!git clone https://github.com/kkahloots/Generative_Models.git # this is for loading git with correct brach
%cd /content/Generative_Models/

/content
fatal: destination path 'Generative_Models' already exists and is not an empty directory.
/content/Generative_Models


In [0]:
os.chdir('/content/Generative_Models/')
print(os.getcwd())

/content/Generative_Models


# Dataset loading

In [0]:
dataset_name = 'dsprites'
import tensorflow_datasets as tfds

info = tfds.builder(dataset_name).info

print(info)

tfds.core.DatasetInfo(
    name='dsprites',
    version=2.0.0,
    description='dSprites is a dataset of 2D shapes procedurally generated from 6 ground truth
independent latent factors. These factors are *color*, *shape*, *scale*,
*rotation*, *x* and *y* positions of a sprite.

All possible combinations of these latents are present exactly once,
generating N = 737280 total images.

### Latent factor values

*   Color: white
*   Shape: square, ellipse, heart
*   Scale: 6 values linearly spaced in [0.5, 1]
*   Orientation: 40 values in [0, 2 pi]
*   Position X: 32 values in [0, 1]
*   Position Y: 32 values in [0, 1]

We varied one latent at a time (starting from Position Y, then Position X, etc),
and sequentially stored the images in fixed order.
Hence the order along the first dimension is fixed and allows you to map back to
the value of the latents corresponding to that image.

We chose the latents values deliberately to have the smallest step changes
while ensuring that all pixel outp

In [0]:
intermediate_dim = 20
inputs_shape=(64, 64, 1) # image shape
batch_size = 32
latents_dim = 6

In [0]:
TRAIN_BUF = 600
TEST_BUF = 100

In [0]:
import tensorflow_datasets as tfds
# Construct a tf.data.Dataset
train_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).shuffle(TRAIN_BUF).batch(batch_size)
try:
    test_ds = tfds.load(name=dataset_name, split=tfds.Split.TEST).shuffle(TEST_BUF).batch(batch_size)
except:
    test_ds = tfds.load(name=dataset_name, split=tfds.Split.TRAIN).shuffle(TEST_BUF).batch(batch_size)

In [0]:
from data.gt_load.datasets import load

In [0]:
#DATA_DOWN_PATH = '..'+sep_local+'..'+sep_local+'..'+sep_local+'data'
DATA_DOWN_PATH = os.getcwd() + sep_local+'data'
Script_dir = os.getcwd() + sep_local+'data'+sep_local+'download_gt_data.sh'
# Script call to download "dsprites_full" dataset_name 
!/bin/bash $Script_dir -f $DATA_DOWN_PATH -d $dataset_name

In [0]:
DATA_PATH =  os.getcwd() + sep_local+'data' +sep_local+'.gt_datasets'
#absolute = abspath(DATA_PATH)
print(DATA_PATH)

/content/Generative_Models/data/.gt_datasets


In [0]:
!ls /content/Generative_Models/data/.gt_datasets/dsprites

dsprites_ndarray_co1sh3sc6or40x32y32_64x64.npz


In [0]:
eval_dataset = load(dataset_name='dsprites_full',dataset_path=DATA_PATH)

In [0]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data['image'].numpy().max())
    break

In [0]:
_instance_scale

1.0

In [0]:
import numpy as np
from collections.abc import Iterable

In [0]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [0]:
_outputs_shape

4096

# Model's Layers definition

In [0]:
menc_lays = [tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latents_dim)]

venc_lays = [tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim//2, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latents_dim)]

dec_lays = [tf.keras.layers.Dense(units=latents_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [0]:
from google.colab import drive
drive.mount('/content/drive')
#!mkdir /content/drive/My\ Drive/DstResults/
MAIN_SAVE_DIR = '/content/drive/My Drive/DstResults'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
model_name = dataset_name+'GAVAE_Dense_reconst_ell'
#experiments_dir='experiments'+sep_local+model_name
recording_dir=MAIN_SAVE_DIR + sep_local  +'recording-Dense'+sep_local + model_name

In [0]:
from training.adversarial_basic.generative_adversarial.autoencoders.VAAE import VAAE as AE

In [0]:
variables_params = \
[
    {
        'name': 'inference_mean', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': menc_lays
    },
    
    {
        'name': 'inference_logvariance', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': venc_lays
    },
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist

In [0]:
_restore = os.path.join(recording_dir, 'var_save_dir')

In [0]:
create_if_not_exist(_restore)
_restore

'/content/drive/My Drive/DstResults/recording-Dense/dspritesGAVAE_Dense_reconst_ell/var_save_dir'

In [0]:
#to restore trained model, set filepath=_restore

In [0]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath=None
    )

  WARNING  | None
  WARNING  | None


Model: "inference_mean"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_mean_inputs (Input [(None, 64, 64, 1)]       0         
_________________________________________________________________
dense (Dense)                (None, 64, 64, 10)        20        
_________________________________________________________________
dense_1 (Dense)              (None, 64, 64, 10)        110       
_________________________________________________________________
flatten (Flatten)            (None, 40960)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 245766    
_________________________________________________________________
batch_normalization (BatchNo (None, 6)                 24        
_________________________________________________________________
dropout (Dropout)            (None, 6)              

  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 6)]               0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_7 (Dense)              (None, 20)                140       
_________________________________________________________________
dense_8 (Dense)              (None, 4096)              86016     
_________________________________________________________________
reshape (Reshape)            (None, 64, 64, 1)         0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 64, 1)         4         
_________________________________________________________________
dropout_2 (Dropout)          (None, 64, 64, 1)         0

In [0]:
from statistical.basic_adversarial_losses import \
    create_generative_discriminator_real_losses, \
    create_generative_discriminator_fake_losses, \
    create_generative_generator_fake_losses
    

generative_discriminator_losses = {
    'generative_discriminator_real_outputs': create_generative_discriminator_real_losses,
    'generative_discriminator_fake_outputs': create_generative_discriminator_fake_losses,
    'generative_generator_fake_outputs': create_generative_generator_fake_losses,
}

In [0]:
#ae.compile(metrics=None)
discr2gen_rate = 0.001
gen2trad_rate = 0.1

ae.compile(
    adversarial_losses=generative_discriminator_losses,
    adversarial_weights={'generator_weight': gen2trad_rate, 'discriminator_weight': discr2gen_rate}
)

Model: "dspritesGAVAE_Dense_reconst_ell"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inference_logvariance_inputs (I [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
inference_logvariance (Model)   (None, 6)            245920      inference_logvariance_inputs[0][0
__________________________________________________________________________________________________
tf_op_layer_mul (TensorFlowOpLa [(None, 6)]          0           inference_logvariance[1][0]      
__________________________________________________________________________________________________
tf_op_layer_Exp (TensorFlowOpLa [(None, 6)]          0           tf_op_layer_mul[0][0]            
____________________________________________________________________

# Callbacks

In [0]:
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver
from training.callbacks.disentangle_supervied import DisentanglementSuperviedMetrics
from training.callbacks.disentangle_unsupervied import DisentanglementUnsuperviedMetrics

In [0]:
gts_csv = os.path.join(recording_dir, 'csv_dir', 'gts_metrics')
gtu_csv = os.path.join(recording_dir, 'csv_dir', 'gtu_metrics')

In [0]:
gts_mertics = DisentanglementSuperviedMetrics(            
    ground_truth_data=eval_dataset,
    representation_fn=lambda x: ae.encode(x),
    random_state=np.random.RandomState(0),
    file_Name=gts_csv,
    num_train=10000,
    num_test=200,
    batch_size=batch_size,
    continuous_factors=False,
    gt_freq=10
)
gtu_mertics = DisentanglementUnsuperviedMetrics(            
    ground_truth_data=eval_dataset,
    representation_fn=lambda x: ae.encode(x),
    random_state=np.random.RandomState(0),
    file_Name=gtu_csv,
    num_train=10000,
    num_test=200,
    batch_size=batch_size,
    gt_freq=10
)

In [0]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [0]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [0]:
csv_dir = os.path.join(recording_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'/content/drive/My Drive/DstResults/recording-Dense/dspritesGAVAE_Dense_reconst_ell/csv_dir/dspritesGAVAE_Dense_reconst_ell.csv'

In [0]:
image_gen_dir = os.path.join(recording_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [0]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=True)

In [0]:
import numpy as np

# Model Training

In [0]:
#!pip install tensorflow==2.1.0-rc1

In [0]:
ae.fit(
    x=train_ds,
    input_kw='image',
    steps_per_epoch=1000,
    epochs=int(1e6), 
    verbose=2,
    callbacks=[ es, ms, csv_log, sg, gts_mertics, gtu_mertics],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=1000
)


training <class 'training.autoencoding_basic.autoencoders.VAE.VAE'>
Train for 1000 steps, validate for 1000 steps
Epoch 1/1000000


/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `nump

1000/1000 - 685s - loss: 1276.8429 - x_logpdf_loss: -1.3412e+00 - x_logits_loss: 1276.4857 - z_latents_loss: 1.6080 - x_logits_psnr: 10.3754 - x_logits_total_variation: 47573.6016 - x_logits_ssim: -4.2429e-04 - x_logits_sharpdiff: 4.2846 - x_logits_mean_absolute_error: 0.1626 - x_logits_mean_squared_error: 0.0929 - val_loss: 19295.6067 - val_x_logpdf_loss: -1.3316e+00 - val_x_logits_loss: 19293.4551 - val_z_latents_loss: 1.6335 - val_x_logits_psnr: 7.3777 - val_x_logits_total_variation: 6158.6934 - val_x_logits_ssim: 7.0039e-05 - val_x_logits_sharpdiff: 12.1721 - val_x_logits_mean_absolute_error: 0.1920 - val_x_logits_mean_squared_error: 0.1865
Epoch 2/1000000
1000/1000 - 62s - loss: 1095.5119 - x_logpdf_loss: -1.4072e+00 - x_logits_loss: 1095.0100 - z_latents_loss: 1.8323 - x_logits_psnr: 10.5899 - x_logits_total_variation: 46090.6797 - x_logits_ssim: -2.5344e-03 - x_logits_sharpdiff: 4.4015 - x_logits_mean_absolute_error: 0.1695 - x_logits_mean_squared_error: 0.0881 - val_loss: 7333.

/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `nump

1000/1000 - 707s - loss: 1014.4848 - x_logpdf_loss: -1.5531e+00 - x_logits_loss: 1012.6334 - z_latents_loss: 3.2672 - x_logits_psnr: 10.7718 - x_logits_total_variation: 45435.5703 - x_logits_ssim: -5.2456e-03 - x_logits_sharpdiff: 4.4448 - x_logits_mean_absolute_error: 0.1688 - x_logits_mean_squared_error: 0.0842 - val_loss: 6618.0015 - val_x_logpdf_loss: -1.5614e+00 - val_x_logits_loss: 6613.0596 - val_z_latents_loss: 2.8063 - val_x_logits_psnr: 13.9586 - val_x_logits_total_variation: 1519.1296 - val_x_logits_ssim: 3.0006e-05 - val_x_logits_sharpdiff: 15.7119 - val_x_logits_mean_absolute_error: 0.0469 - val_x_logits_mean_squared_error: 0.0449


/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,


Epoch 12/1000000
1000/1000 - 64s - loss: 1012.2316 - x_logpdf_loss: -1.5655e+00 - x_logits_loss: 1010.2767 - z_latents_loss: 3.3790 - x_logits_psnr: 10.7775 - x_logits_total_variation: 45433.0039 - x_logits_ssim: -5.3244e-03 - x_logits_sharpdiff: 4.4446 - x_logits_mean_absolute_error: 0.1685 - x_logits_mean_squared_error: 0.0841 - val_loss: 7497.7382 - val_x_logpdf_loss: -1.5478e+00 - val_x_logits_loss: 7494.0586 - val_z_latents_loss: 2.9170 - val_x_logits_psnr: 8.9409 - val_x_logits_total_variation: 5589.7056 - val_x_logits_ssim: 9.5575e-05 - val_x_logits_sharpdiff: 12.4651 - val_x_logits_mean_absolute_error: 0.1434 - val_x_logits_mean_squared_error: 0.1320
Epoch 13/1000000
1000/1000 - 64s - loss: 1011.2391 - x_logpdf_loss: -1.5717e+00 - x_logits_loss: 1009.1709 - z_latents_loss: 3.4940 - x_logits_psnr: 10.7815 - x_logits_total_variation: 45472.1875 - x_logits_ssim: -5.3490e-03 - x_logits_sharpdiff: 4.4408 - x_logits_mean_absolute_error: 0.1683 - x_logits_mean_squared_error: 0.0840 - 

/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `nump

1000/1000 - 676s - loss: 1000.1995 - x_logpdf_loss: -1.6425e+00 - x_logits_loss: 997.2558 - z_latents_loss: 4.4123 - x_logits_psnr: 10.8189 - x_logits_total_variation: 45537.9609 - x_logits_ssim: -5.5139e-03 - x_logits_sharpdiff: 4.4340 - x_logits_mean_absolute_error: 0.1668 - x_logits_mean_squared_error: 0.0833 - val_loss: 3092.2246 - val_x_logpdf_loss: -1.6544e+00 - val_x_logits_loss: 3089.1797 - val_z_latents_loss: 3.7089 - val_x_logits_psnr: 13.9162 - val_x_logits_total_variation: 1349.3569 - val_x_logits_ssim: 1.6402e-04 - val_x_logits_sharpdiff: 16.0219 - val_x_logits_mean_absolute_error: 0.0509 - val_x_logits_mean_squared_error: 0.0457
Epoch 22/1000000
1000/1000 - 64s - loss: 999.5872 - x_logpdf_loss: -1.6503e+00 - x_logits_loss: 996.5461 - z_latents_loss: 4.5135 - x_logits_psnr: 10.8204 - x_logits_total_variation: 45551.3945 - x_logits_ssim: -5.5359e-03 - x_logits_sharpdiff: 4.4324 - x_logits_mean_absolute_error: 0.1667 - x_logits_mean_squared_error: 0.0833 - val_loss: 858.1804

/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `numpy.expand_dims` function is not implemented by Dask array. You may want to use the da.map_blocks function or something similar to silence this warning. Your code may stop working in a future release.
  FutureWarning,
/usr/local/lib/python3.6/dist-packages/dask/array/core.py:1361: FutureWarning: The `nump

# Model Evaluation

## inception_score

In [0]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [0]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=200)
print(f'inception_score mean: {is_mean}, sigma: {is_sigma}')

## Frechet_inception_distance

In [0]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [0]:
fis_score = frechet_inception_distance(ae, training_generator, tolerance_threshold=1e-6, max_iteration=10, batch_size=32)
print(f'frechet inception distance: {fis_score}')

## perceptual_path_length_score

In [0]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [0]:
ppl_mean_score = perceptual_path_length_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200, batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

## precision score

In [0]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [0]:
_precision_score = precision_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'precision score: {_precision_score}')

## recall score

In [0]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [0]:
_recall_score = recall_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=200)
print(f'recall score: {_recall_score}')

# Image Generation

## image reconstruction

### Training dataset

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, training_generator, save_dir)

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, testing_generator, save_dir)

## with Randomness

In [0]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, training_generator, save_dir)

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, testing_generator, save_dir)

### Complete Randomness

In [0]:
from training.generators.image_generation_testing import generate_images_randomly

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

In [0]:
from training.generators.image_generation_testing import interpolate_a_batch

In [0]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, testing_generator, save_dir)